In [1]:
import os
from chainbase_sdk.sql_alpha import ChainbaseSQLAlpha

CHAINBASE_API_KEY=os.environ['CHAINBASE_API_KEY']

L2_CHAIN = 'optimism'
L1_PORTAL = '0x25ace71c97B33Cc4729CF772ae268934F7ab5fA1'
#L1_PORTAL = '0x49048044d57e1c92a77f79988d21fa8faf74e97e'

In [2]:
client = ChainbaseSQLAlpha(api_key=CHAINBASE_API_KEY)

In [3]:
def withdrawal_basic_query(client, chain, contract_address):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) <> 0
),
proven_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
aggregate_data AS (
    SELECT 
        iw.block_number AS initiated_block_number,
        iw.block_timestamp AS initiated_block_timestamp,
        iw.transaction_hash AS initiated_transaction_hash,
        iw.sender,
        iw.target,
        iw.value,
        iw.withdrawal_hash AS initiated_withdrawal_hash,
        pw.block_number AS proven_block_number,
        pw.block_timestamp AS proven_block_timestamp,
        pw.transaction_hash AS proven_transaction_hash,
        pw.withdrawal_hash AS proven_withdrawal_hash,
        fw.block_number AS finalized_block_number,
        fw.block_timestamp AS finalized_block_timestamp,
        fw.transaction_hash AS finalized_transaction_hash,
        fw.withdrawal_hash AS finalized_withdrawal_hash
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
)
SELECT * FROM aggregate_data
    """
    # missing and success = true

    df_l1_finalized = client.query_pandas(query)

    return df_l1_finalized

In [4]:
def deposit_basic_query(client, contract_address):
    query = f"""
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS target,
        CAST(CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS BIGINT) AS value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{contract_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
    """
    df_deposit = client.query_pandas(query)

    return df_deposit

In [5]:
base_withdrawals = withdrawal_basic_query(client=client, chain=L2_CHAIN,contract_address=L1_PORTAL)

len(base_withdrawals)

10000

In [6]:
base_withdrawals.head()

,initiated_block_number,initiated_block_timestamp,initiated_transaction_hash,sender,target,value,initiated_withdrawal_hash,proven_block_number,proven_block_timestamp,proven_transaction_hash,proven_withdrawal_hash,finalized_block_number,finalized_block_timestamp,finalized_transaction_hash,finalized_withdrawal_hash
0,5599150,2023-10-22 15:14:07,0xa2b46d1cc5b2a4fb2c5f192ea4e246a9e2827a142f00...,0xb8372df9d06345eb96e27d0ec87341b10f23a6c7,0xb8372df9d06345eb96e27d0ec87341b10f23a6c7,100000000000000000,0x4b76bdd7bb92501906a229e4cb13ee47534c700608ab...,18440048,2023-10-27 07:27:23,0x3ee61ca29a864266203c01fa02059cc94e811802a684...,0x4b76bdd7bb92501906a229e4cb13ee47534c700608ab...,18496113,2023-11-04 03:50:47,0xd80bbcdeb5f81ca3d6646067962ddd1636f7ef0ca299...,0x4b76bdd7bb92501906a229e4cb13ee47534c700608ab...
1,4997931,2023-10-08 17:13:29,0x4e697c6162dd66c1741469c03ae384984cfadd94ac04...,0x991e0876716df78c9252109c9b16975a08f892d1,0x991e0876716df78c9252109c9b16975a08f892d1,1320000000000000000,0xd34441b9ac74a6962d8b41fc2c8134fa24114cc01802...,18307539,2023-10-08 18:27:23,0x5fc9efdaeae1567280257b76d88d7c9f502688d547a3...,0xd34441b9ac74a6962d8b41fc2c8134fa24114cc01802...,18497907,2023-11-04 09:53:47,0xe4403cf84d0de1dc522bfa56295bb75253266bff614a...,0xd34441b9ac74a6962d8b41fc2c8134fa24114cc01802...
2,5767776,2023-10-26 12:54:59,0x69b3749484ede234284eb1c81bac7c2d005d04d59459...,0xb3c7060d67f373e1878bd8daee192ebbe7a797b5,0xb3c7060d67f373e1878bd8daee192ebbe7a797b5,86599000000000000,0x606f5428305d087af5cde856be3bdeb28ac03a9d92ed...,18452126,2023-10-28 23:59:47,0x27008483b2f0ed6bf4d5f5dc04c1d9fdcd5e98bac236...,0x606f5428305d087af5cde856be3bdeb28ac03a9d92ed...,18502120,2023-11-05 00:04:23,0x1acd8104ff62564548d988e080df71ae2840e45a32ac...,0x606f5428305d087af5cde856be3bdeb28ac03a9d92ed...
3,5676588,2023-10-24 10:15:23,0xfbd152d1432dd71b646d7750eaa5c0d290ae4e767908...,0x28133ef84b61c0d6e08c0b189554e99bd30c32e2,0x28133ef84b61c0d6e08c0b189554e99bd30c32e2,30000000000000000,0x9d258acafa0de1b89e2f905b5b8266de9f38d10ee161...,18452972,2023-10-29 02:49:47,0xe597f495063353950201eebebd2eb5bb258dac24b2fe...,0x9d258acafa0de1b89e2f905b5b8266de9f38d10ee161...,18503191,2023-11-05 03:40:23,0x6f97ce8df3e97b4dd45aeb0f1697fbb92decf2246789...,0x9d258acafa0de1b89e2f905b5b8266de9f38d10ee161...
4,5710309,2023-10-25 04:59:25,0x6079e1e385e9b556c0e18f21f02f2a9b9f108f7f2350...,0xb3d98dd918a6cec01f661702a7feb40178abd756,0xb3d98dd918a6cec01f661702a7feb40178abd756,30000000000000000,0x5936e58128a18915e6049c0d96830db794e0fd1a2ca0...,18452702,2023-10-29 01:55:47,0x10f2e9f260695820ed5f9712c86725a7cf92e3d2c553...,0x5936e58128a18915e6049c0d96830db794e0fd1a2ca0...,18503448,2023-11-05 04:32:35,0x34738c6fc373756cfbe097f73dcd0cd1019b910a7e19...,0x5936e58128a18915e6049c0d96830db794e0fd1a2ca0...


In [7]:
base_deposits = deposit_basic_query(client=client, contract_address=L1_PORTAL)

len(base_deposits)

10000

In [20]:
base_deposits.head()

,block_number,block_timestamp,transaction_hash,sender,target,value
0,20802391,2024-09-22 00:00:23,0xe288e69780782a0ec84fe7618c795e7b156fc4c20561...,0x4b34f943181408eac424116af7b7790c94cba8b6,0x3ac2b58084d4b555a2ea19903ee6af820e93180a,31979992589368970
1,20802401,2024-09-22 00:02:23,0x78c73cee168ff0bc99c1e0421bfa7e31b867d7d3bb46...,0x4b34f943181408eac424116af7b7790c94cba8b6,0x5a25c73a1ace243da2ae0c882601510421e45599,2676715822640810
2,20802401,2024-09-22 00:02:23,0x770050b7273724186c8fc44ff40a0859d063e0270d4f...,0x4b34f943181408eac424116af7b7790c94cba8b6,0xbc5029d1fe3e67a6c7366d773fa15dcb1eaeb875,44357005060904840
3,20802416,2024-09-22 00:05:23,0x8512fac67b5b06b6b78fa7d9cd25fbc44a3fc77702e2...,0x4b34f943181408eac424116af7b7790c94cba8b6,0x24b92365cf22da6160752891736627d67da24fc1,1337485908630606
4,20802430,2024-09-22 00:08:11,0x6ef2018390540e2207389aa1aedd4389ff5f2893d259...,0x4b34f943181408eac424116af7b7790c94cba8b6,0x58e81c38d003bb8d7c78433ee668fbfa2508a5c8,19106941551865790


In [9]:
# save to csv

base_withdrawals.to_csv(f'../../data/{L2_CHAIN}/basic_withdrawals.csv', index=False)
base_deposits.to_csv(f'../../data/{L2_CHAIN}/basic_deposits.csv', index=False)

## tables for the frontend
as the api limits to 10_000 rows, it is necessary for me to manipulate the data directly within sql. hence i have made three queries to extract all views i am interested in for each chain.

In [10]:
def counters(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE)/1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash, 
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS withdrawal_hash 
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
aggregate_data AS (
    SELECT 
        iw.block_number AS initiated_block_number,
        iw.block_timestamp AS initiated_block_timestamp,
        iw.transaction_hash AS initiated_transaction_hash,
        iw.sender,
        iw.target,
        iw.value,
        iw.withdrawal_hash AS initiated_withdrawal_hash,
        pw.block_number AS proven_block_number,
        pw.block_timestamp AS proven_block_timestamp,
        pw.transaction_hash AS proven_transaction_hash,
        pw.withdrawal_hash AS proven_withdrawal_hash,
        fw.block_number AS finalized_block_number,
        fw.block_timestamp AS finalized_block_timestamp,
        fw.transaction_hash AS finalized_transaction_hash,
        fw.withdrawal_hash AS finalized_withdrawal_hash
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
)
SELECT
    SUM(value) AS total_value,
    SUM(IF(finalized_transaction_hash IS NULL, value, 0)) AS sum_if_finalized_null,
    SUM(IF(proven_withdrawal_hash IS NULL, value, 0)) AS sum_if_proven_null,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 14 and proven_withdrawal_hash is null, value, 0)) AS not_proven_2w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 28 and proven_withdrawal_hash is null, value, 0)) AS not_proven_4w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 14 AND finalized_transaction_hash IS NULL, value, 0)) AS not_finalized_2w,
    SUM(IF(DATEDIFF(NOW(), initiated_block_timestamp) > 28 AND finalized_transaction_hash IS NULL, value, 0)) AS not_finalized_4w    
FROM 
    aggregate_data    
"""
    df = client.query_pandas(query)
    return df

In [11]:
df_counters = counters(chain=L2_CHAIN, l1_portal_address=L1_PORTAL)

NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE


In [12]:
df_counters

,total_value,sum_if_finalized_null,sum_if_proven_null,not_proven_2w,not_proven_4w,not_finalized_2w,not_finalized_4w
0,17663.807784,2706.992636,1569.406476,1540.777173,1531.944621,2601.398476,2581.154698


In [13]:
df_counters.to_csv(f'../../data/{L2_CHAIN}/counters.csv', index=False)

In [14]:
def daily_volumes(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) / 1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE 
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded pw ON iw.withdrawal_hash = CAST(CAST(pw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        pw.contract_address = '{l1_portal_address}' 
        AND pw.method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded fw ON iw.withdrawal_hash = CAST(CAST(fw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        fw.contract_address = '{l1_portal_address}' 
        AND fw.method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
deposits_basic AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[0]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS target,
        CAST(CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) AS DOUBLE) / 1e18 AS value
    FROM 
        ethereum.transaction_logs_decoded 
    WHERE 
        contract_address = '{l1_portal_address}' 
        AND method_id = '0xb3813568d9991fc951961fcb4c784893574240a28925604d09fc577c55bb7c32' -- TransactionDeposited
        AND CONV(SUBSTR(CAST(decoded AS JSON) -> '$[3]', 3, 64), 16, 10) <> 0 -- exclude zero value messages
),
aggregated_data AS (
    SELECT 
        DATE(iw.block_timestamp) AS date,
        SUM(iw.value) AS initiated_volume,
        COALESCE(SUM(pw.initiated_value), 0) AS proved_volume,
        COALESCE(SUM(fw.initiated_value), 0) AS finalized_volume,
        COALESCE(SUM(dep.value), 0) AS deposited_volume
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
    LEFT JOIN 
        deposits_basic dep ON DATE(iw.block_timestamp) = DATE(dep.block_timestamp)
    GROUP BY 
        DATE(iw.block_timestamp)
),
cumulative_data AS (
    SELECT 
        date,
        initiated_volume,
        proved_volume,
        finalized_volume,
        deposited_volume,
        (COALESCE(deposited_volume, 0) - COALESCE(initiated_volume, 0)) AS daily_difference,
        SUM(initiated_volume) OVER (ORDER BY date) AS cumulative_initiated,
        SUM(proved_volume) OVER (ORDER BY date) AS cumulative_proved,
        SUM(finalized_volume) OVER (ORDER BY date) AS cumulative_finalized,
        SUM(deposited_volume) OVER (ORDER BY date) AS cumulative_deposited
    FROM 
        aggregated_data
)
SELECT 
    date,
    initiated_volume,
    proved_volume,
    finalized_volume,
    deposited_volume,
    daily_difference,
    cumulative_initiated,
    cumulative_proved,
    cumulative_finalized,
    cumulative_deposited
FROM 
    cumulative_data
ORDER BY 
    date
 
"""
    df = client.query_pandas(query)
    return df

In [15]:
df_daily = daily_volumes(chain=L2_CHAIN, l1_portal_address='0x49048044d57e1c92a77f79988d21fa8faf74e97e')

NOT EXISTS:  DATE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE
NOT EXISTS:  DOUBLE


In [16]:
print(df_daily)
df_daily.to_csv(f'../../data/{L2_CHAIN}/daily_volumes.csv', index=False)

           date  initiated_volume  proved_volume  finalized_volume  \
0    2023-07-11          0.370370       0.000000          0.000000   
1    2023-07-12          0.023457       0.023457          0.023457   
2    2023-07-27          2.272500       0.000000          0.000000   
3    2023-07-28         15.614600       0.000000          0.000000   
4    2023-07-30       2451.510515     947.756000        462.320000   
..          ...               ...            ...               ...   
388  2024-10-14       5440.117469    4430.809320          0.000000   
389  2024-10-15       1133.981422     773.425124          0.000000   
390  2024-10-16       4552.635670    4308.761005          0.000000   
391  2024-10-17        751.392978     718.962790          0.000000   
392  2024-10-18       1460.200180     254.426661          0.000000   

     deposited_volume  daily_difference  cumulative_initiated  \
0            0.750000          0.379630          3.703704e-01   
1            0.000000        

In [17]:
def stuck_in_contract(chain, l1_portal_address, client = client):
    query = f"""
WITH initiated_withdrawals AS (
    SELECT 
        block_number, 
        block_timestamp, 
        transaction_hash,
        CAST(CAST(decoded AS JSON) -> '$[1]' AS VARCHAR) AS sender,
        CAST(CAST(decoded AS JSON) -> '$[2]' AS VARCHAR) AS target,
        CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) / 1e18 AS value,
        CAST(CAST(decoded AS JSON) -> '$[6]' AS VARCHAR) AS withdrawal_hash
    FROM 
        {chain}.transaction_logs_decoded
    WHERE
        contract_address = '0x4200000000000000000000000000000000000016'
        AND CAST(CAST(CAST(CAST(decoded AS JSON) -> '$[3]' AS VARCHAR) AS BIGINT) AS DOUBLE) <> 0
),
proven_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded pw ON iw.withdrawal_hash = CAST(CAST(pw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        pw.contract_address = '{l1_portal_address}' 
        AND pw.method_id = '0x67a6208cfcc0801d50f6cbe764733f4fddf66ac0b04442061a8a8c0cb6b63f62' -- WithdrawalProven
),
finalized_withdrawals AS (
    SELECT 
        iw.block_timestamp, 
        iw.withdrawal_hash,
        iw.value AS initiated_value
    FROM 
        initiated_withdrawals iw
    JOIN 
        ethereum.transaction_logs_decoded fw ON iw.withdrawal_hash = CAST(CAST(fw.decoded AS JSON) -> '$[0]' AS VARCHAR)
    WHERE 
        fw.contract_address = '{l1_portal_address}' 
        AND fw.method_id = '0xdb5c7652857aa163daadd670e116628fb42e869d8ac4251ef8971d9e5727df1b' -- WithdrawalFinalized
),
withdrawals_not_withdrawn AS (
    SELECT 
        iw.withdrawal_hash,
        iw.value AS initiated_value,
        iw.block_timestamp
    FROM 
        initiated_withdrawals iw
    LEFT JOIN 
        proven_withdrawals pw ON iw.withdrawal_hash = pw.withdrawal_hash
    LEFT JOIN 
        finalized_withdrawals fw ON iw.withdrawal_hash = fw.withdrawal_hash
    WHERE 
        pw.withdrawal_hash IS NULL 
        AND fw.withdrawal_hash IS NULL
        AND iw.block_timestamp < DATE_SUB(CURRENT_DATE, INTERVAL 14 DAY)
),
daily_not_withdrawn AS (
    SELECT 
        DATE(block_timestamp) AS date,
        SUM(initiated_value) AS total_not_withdrawn
    FROM 
        withdrawals_not_withdrawn
    GROUP BY 
        DATE(block_timestamp)
)
SELECT 
    date,
    COALESCE(total_not_withdrawn, 0) AS total_not_withdrawn
FROM 
    daily_not_withdrawn
ORDER BY 
    date
"""

    df = client.query_pandas(query)
    return df

In [18]:
df_stuck = stuck_in_contract(chain=L2_CHAIN, l1_portal_address=L1_PORTAL)

NOT EXISTS:  DATE
NOT EXISTS:  DOUBLE


In [19]:
df_stuck.to_csv(f'../../data/{L2_CHAIN}/forgotten_funds.csv', index=False)